In [1]:
from icecream import ic

In [2]:
from lib.networks.pvnet.resnet import resnet50
resnet50_8s = resnet50(fully_conv=True,
                   pretrained=True,
                   output_stride=8,
                   remove_avg_pool_layer=True)

In [3]:
resnet50_8s 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
import torch 
import torch.nn as nn

torch.cuda.empty_cache()


In [4]:
class Resnet50(nn.Module):
    def __init__(self, ver_dim, seg_dim, l4=2048, l3=512, l2=256, l1=64, l0=32,raw_dim=30):
        super(Resnet50, self).__init__()
        
        
        self.ver_dim = ver_dim
        self.seg_dim = seg_dim 
        
        self.resnet50_8s  = resnet50_8s
        
        resnet50_8s.fc = nn.Sequential(
            nn.Conv2d(l4, l3, 3 , 1, 1, bias = False),
            nn.BatchNorm2d(l3),
            nn.ReLU(True)
        )
        
        self.conv8s = nn.Sequential(
            nn.Conv2d(l3 + l3 , l2, 3,1,1, bias =False),
            nn.BatchNorm2d(l2),
            nn.ReLU(True)
        )
        
        self.up_1 = nn.UpsamplingBilinear2d(scale_factor=2)
        
        self.conv4s = nn.Sequential(
            nn.Conv2d(l2 + l2 , l1, 3,1,1, bias =False),
            nn.BatchNorm2d(l1),
            nn.ReLU(True)
        )
                
        self.conv2s = nn.Sequential(
            nn.Conv2d(l1 + l1 , l0, 3,1,1, bias =False),
            nn.BatchNorm2d(l0),
            nn.ReLU(True)
        )

        self.up_2 = nn.UpsamplingBilinear2d(scale_factor=2)
        
        self.conv1s = nn.Sequential(
            nn.Conv2d(l0 + 3 , raw_dim, 3,1,1, bias =False),
            nn.BatchNorm2d(raw_dim),
            nn.LeakyReLU(0.1, True),
            nn.Conv2d(raw_dim, seg_dim + ver_dim , 3, 1, 1, bias=False)
        )

        self.up_3 = nn.UpsamplingBilinear2d(scale_factor=2)
        
        
    def forward(self,x):
    
        x2s, x4s, x8s, x16s, x32s, xfc = self.resnet50_8s(x)
        xi = self.conv8s(torch.cat([xfc, x8s],1))
        xi = self.up_1(xi)
        xi = self.conv4s(torch.cat([xi, x4s],1))
        xi = self.up_2(xi)
        xi = self.conv2s(torch.cat([xi, x2s],1))
        xi = self.up_3(xi)
        xi = self.conv1s(torch.cat([xi, x],1))


        return xi

In [5]:
model = Resnet50(18,2).to('cuda:0')

In [6]:
dummy_input = torch.rand(1,3,640,480).to('cuda:0')
model(dummy_input).shape

torch.Size([1, 20, 640, 480])

In [9]:
torch.onnx.export(model, dummy_input, "fd.onnx",opset_version=11)

In [11]:
torch.jit.trace(model, dummy_input)

Resnet50(
  original_name=Resnet50
  (resnet50_8s): ResNet(
    original_name=ResNet
    (conv1): Conv2d(original_name=Conv2d)
    (bn1): BatchNorm2d(original_name=BatchNorm2d)
    (relu): ReLU(original_name=ReLU)
    (maxpool): MaxPool2d(original_name=MaxPool2d)
    (layer1): Sequential(
      original_name=Sequential
      (0): Bottleneck(
        original_name=Bottleneck
        (conv1): Conv2d(original_name=Conv2d)
        (bn1): BatchNorm2d(original_name=BatchNorm2d)
        (conv2): Conv2d(original_name=Conv2d)
        (bn2): BatchNorm2d(original_name=BatchNorm2d)
        (conv3): Conv2d(original_name=Conv2d)
        (bn3): BatchNorm2d(original_name=BatchNorm2d)
        (relu): ReLU(original_name=ReLU)
        (downsample): Sequential(
          original_name=Sequential
          (0): Conv2d(original_name=Conv2d)
          (1): BatchNorm2d(original_name=BatchNorm2d)
        )
      )
      (1): Bottleneck(
        original_name=Bottleneck
        (conv1): Conv2d(original_name=Con

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 320, 240]           9,408
       BatchNorm2d-2         [-1, 64, 320, 240]             128
              ReLU-3         [-1, 64, 320, 240]               0
         MaxPool2d-4         [-1, 64, 160, 120]               0
            Conv2d-5         [-1, 64, 160, 120]           4,096
       BatchNorm2d-6         [-1, 64, 160, 120]             128
              ReLU-7         [-1, 64, 160, 120]               0
            Conv2d-8         [-1, 64, 160, 120]          36,864
       BatchNorm2d-9         [-1, 64, 160, 120]             128
             ReLU-10         [-1, 64, 160, 120]               0
           Conv2d-11        [-1, 256, 160, 120]          16,384
      BatchNorm2d-12        [-1, 256, 160, 120]             512
           Conv2d-13        [-1, 256, 160, 120]          16,384
      BatchNorm2d-14        [-1, 256, 1

In [13]:
from torch.utils.mobile_optimizer import optimize_for_mobile
model.eval()
example = torch.rand(1,3,640,480).to('cuda')
traced_script_module = torch.jit.trace(model, example)
traced_script_module_optimized = optimize_for_mobile(traced_script_module)
traced_script_module_optimized._save_for_lite_interpreter("model.ptl")